In [ ]:
import os
from dotenv import load_dotenv
import logging
from autogen_agentchat import EVENT_LOGGER_NAME
from autogen_agentchat.agents import CodingAssistantAgent, ToolUseAssistantAgent
from autogen_agentchat.logging import ConsoleLogHandler
from autogen_agentchat.teams import MaxMessageTermination, RoundRobinGroupChat, StopMessageTermination
from autogen_core.components.models import AzureOpenAIChatCompletionClient, OpenAIChatCompletionClient
from autogen_core.components.tools import FunctionTool

load_dotenv(dotenv_path=r"C:\Users\hyssh\workspace\azure-openai-quickstart\quickstart-learnfast\autogen-demo\.env", override=True)

logger = logging.getLogger(EVENT_LOGGER_NAME)
logger.addHandler(ConsoleLogHandler())
logger.setLevel(logging.INFO)


# define a tool
async def get_weather(city: str) -> str:
    return f"The weather in {city} is 70 degrees and Sunny."


# wrap the tool for use with the agent
get_weather_tool = FunctionTool(get_weather, description="Get the weather for a city")

az_model_client = AzureOpenAIChatCompletionClient(
    model=os.getenv("DEPLOYMENT_NAME"),
    api_version="2024-06-01",
    azure_eenndpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"), 
    model_capabilities={
        "vision": True,
        "function_calling": True,
        "json_output": True,
    },
)


# define an agent
weather_agent = ToolUseAssistantAgent(
    name="writing_agent",
    model_client=az_model_client,
    registered_tools=[get_weather_tool],
)

# add the agent to a team
agent_team = RoundRobinGroupChat([weather_agent])

result = await agent_team.run(
    task="What is the weather in New York?",
    termination_condition=MaxMessageTermination(max_messages=1),
)

print("\n", result)

In [ ]:
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json

# Load LLM inference endpoints from an env variable or a file
# See https://microsoft.github.io/autogen/docs/FAQ#set-your-api-endpoints
# and OAI_CONFIG_LIST_sample
config_list = config_list_from_json(env_or_file=r"C:\Users\hyssh\workspace\azure-openai-quickstart\quickstart-learnfast\autogen-demo\OAI_CONFIG_LIST")

assistant = AssistantAgent("assistant", llm_config={"config_list": config_list})

user_proxy = UserProxyAgent(
    "user_proxy", code_execution_config={"work_dir": "coding", "use_docker": False}
)  # IMPORTANT: set to True to run code in docker, recommended

sample_question = """
Use --data-- to train a regression model to predict y when x is 9?

--data--  
X	Y  
2	6  
4	12  
6	18  
8	24  
10	30  
12	36  
14	42  
16	48  
18	54  
20	60  
22	66  
----
"""

user_proxy.initiate_chat(assistant, message=sample_question)



